## python-can-demo

In [1]:
!pip install python-can uptime
# or use !pip install python-can<4 uptime for v3 since v4 is under development

((Or see instructions at:https://python-can.readthedocs.io/en/master/index.html))

## Setup to use with PCAN-Usb Pro

In [2]:
import can
import time

In [3]:
config = {'interface': 'pcan', 'bitrate': 125000}
can.rc.update(config)

## List all available channels.

In [4]:
can.detect_available_configs('pcan')

[{'interface': 'pcan', 'channel': 'PCAN_USBBUS1', 'supports_fd': True},
 {'interface': 'pcan', 'channel': 'PCAN_USBBUS2', 'supports_fd': True}]

## Connect to a bus, then flash the lights for 10 seconds

In [5]:
with can.Bus(channel="PCAN_USBBUS1") as  bus:    
    print(f'{bus.status_string()=}')
    
    # display device number
    print(f'{bus.get_device_number()=}')
    
    # flash the lights on the device 
    bus.flash(True)
    time.sleep(10)
    bus.flash(False)
    
    # get the internal device number 
    print(f'{bus.get_device_number()}')
    
    # change the internal device number to something useful (like the number taped on the back of the device)
    # bus.set_device_number(20277) 
    # print(f'{bus.get_device_number()}')
    

bus.status_string()='OK'
bus.get_device_number()=20277
20277


## Send and receive a single message

In [6]:
with can.Bus(channel="PCAN_USBBUS1") as  bus_a, can.Bus(channel="PCAN_USBBUS2") as bus_b:
    try:
        message = can.Message(arbitration_id=15)
        bus_a.send(message)
        print(f'Sent {repr(message)} on {bus_a.channel_info}')
    except can.CanError:
        print("NOT sent")
        
    received_msg = bus_b.recv()
    print('Received', repr(received_msg), 'on', bus_b.channel_info)
    

Sent can.Message(timestamp=0.0, arbitration_id=0xf, is_extended_id=True, dlc=0, data=[]) on PCAN_USBBUS1
Received can.Message(timestamp=1664198156.194402, arbitration_id=0xf, is_extended_id=True, dlc=0, data=[]) on PCAN_USBBUS2


## Send mulitiple messsages and attach a listener/notifier

In [7]:
with can.Bus(channel="PCAN_USBBUS1") as  bus_a, can.Bus(channel="PCAN_USBBUS2") as bus_b:
    print(f'{bus_a.status_string()=}, {bus_b.status_string()=}')
    notify =  can.Notifier(bus_b, [can.Printer()])
    
    for i in range(5):
        # send a couple of messages
        bus_a.send(can.Message(data=[i] * 3))
        
    
    # give the notifier some time to notify
    time.sleep(2)
    notify.stop()

bus_a.status_string()='OK', bus_b.status_string()='OK'
Timestamp: 1664198156.593659    ID: 00000000    X Rx                DL:  3    00 00 00
Timestamp: 1664198156.594459    ID: 00000000    X Rx                DL:  3    01 01 01
Timestamp: 1664198156.595267    ID: 00000000    X Rx                DL:  3    02 02 02
Timestamp: 1664198156.596075    ID: 00000000    X Rx                DL:  3    03 03 03
Timestamp: 1664198156.596875    ID: 00000000    X Rx                DL:  3    04 04 04


See can.interfaces.pcan.pcan.

## Accessing the PCAN Basic API from Peak
Pyton-can acts as a higher-level wrapper around the lower-level but official PCAN Basic API from Peak. All the methods from the Peak API are accessible thorugh here. 

In [8]:
from pprint import pprint
with can.Bus(channel="PCAN_USBBUS1") as  bus, can.Bus(channel="PCAN_USBBUS2") as  bus_2:
    
    print(bus.get_api_version())
    print('The backend:', type(bus.m_objPCANBasic).__name__)
    print('...', help(bus.m_objPCANBasic))

4.6.1.728
The backend: PCANBasic
Help on PCANBasic in module can.interfaces.pcan.basic object:

class PCANBasic(builtins.object)
 |  PCAN-Basic API class implementation
 |  
 |  Methods defined here:
 |  
 |  FilterMessages(self, Channel, FromID, ToID, Mode)
 |      Configures the reception filter
 |      
 |      Remarks:
 |        The message filter will be expanded with every call to this function.
 |        If it is desired to reset the filter, please use the 'SetValue' function.
 |      
 |      Parameters:
 |        Channel : A TPCANHandle representing a PCAN Channel
 |        FromID  : A c_uint value with the lowest CAN ID to be received
 |        ToID    : A c_uint value with the highest CAN ID to be received
 |        Mode    : A TPCANMode representing the message type (Standard, 11-bit
 |                  identifier, or Extended, 29-bit identifier)
 |      
 |      Returns:
 |        A TPCANStatus error code
 |  
 |  GetErrorText(self, Error, Language=0)
 |      Configures or

In [9]:
help(bus.m_objPCANBasic)

Help on PCANBasic in module can.interfaces.pcan.basic object:

class PCANBasic(builtins.object)
 |  PCAN-Basic API class implementation
 |  
 |  Methods defined here:
 |  
 |  FilterMessages(self, Channel, FromID, ToID, Mode)
 |      Configures the reception filter
 |      
 |      Remarks:
 |        The message filter will be expanded with every call to this function.
 |        If it is desired to reset the filter, please use the 'SetValue' function.
 |      
 |      Parameters:
 |        Channel : A TPCANHandle representing a PCAN Channel
 |        FromID  : A c_uint value with the lowest CAN ID to be received
 |        ToID    : A c_uint value with the highest CAN ID to be received
 |        Mode    : A TPCANMode representing the message type (Standard, 11-bit
 |                  identifier, or Extended, 29-bit identifier)
 |      
 |      Returns:
 |        A TPCANStatus error code
 |  
 |  GetErrorText(self, Error, Language=0)
 |      Configures or sets a PCAN Channel value
 |    